# Ben Needs a Friend - Retrieval Augmented Generation (RAG)
This is part of the "Ben Needs a Friend" tutorial.  See all the notebooks and materials [here](https://github.com/bpben/ben_friend).

In this notebook, we set up an approach to use a set of documents ("memories") in a Retrieval Augmented Generation (RAG) workflow.

This notebook is intended to be run in Kaggle Notebooks with GPU acceleration.  Access that version [here](https://www.kaggle.com/code/bpoben/ben-needs-a-friend-rag). 

If you want to run this locally, edit the `model_name` path.  Note that this assumes use of GPUs, it may be slow or not work at all if you do not have access to GPUs.

In [1]:
from llamabot import SimpleBot, StructuredBot, ChatBot
import json
from pydantic import BaseModel
import tempfile

sft_model = "qwen2.5:1.5b"

### Vector stores
The first part of RAG is "retrieval".  To do that we essentially need to create a mechanism for the model to retrieve relevant information.  One approach is to create a set of "embeddings" for our each memory I have with my AI friend that can be compared against the input prompt.

#### LanceDB implementation
One approach to setting up this vector store is to use [LanceDB's implementation of embedding](https://lancedb.github.io/lancedb/embeddings/embedding_functions/).  Llamabot uses this by default.  Below is an overview of what happens under the hood, but we'll just rely on Llamabot's implementation.

In [2]:
import lancedb
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

# create a database
db = lancedb.connect("/tmp/db")
db.drop_all_tables()
# initialize a default sentence-transformers model (paraphrase-MiniLM-L6-v2)
model = get_registry().get("sentence-transformers").create()

# specify a schema (just text + vector)
class Words(LanceModel):
    text: str = model.SourceField()
    vector: Vector(model.ndims()) = model.VectorField()


try:
    table = db.create_table("words", schema=Words)
except ValueError:
    table = db.open_table("words")

# add in some entries
table.add(
    [
        {"text": "hello world"},
        {"text": "goodbye world"}
    ]
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
# look at the entries
table.head()

pyarrow.Table
text: string not null
vector: fixed_size_list<item: float>[384]
  child 0, item: float
----
text: [["hello world","goodbye world"]]
vector: [[[-0.034477282,0.031023193,0.0067349547,0.026109016,-0.039362084,...,0.033231944,0.023792244,-0.022889705,0.038937513,0.030206805],[0.022745548,0.080096945,0.033089288,0.0010678794,0.04239042,...,0.013829722,0.09334315,-0.011528731,-0.01818858,-0.041075554]]]

In [4]:
query = "greetings"
search_query = table.search(query)
search_query._query[:10]

array([-0.09538583,  0.13055924,  0.08160689,  0.07081324, -0.01289312,
       -0.10308413,  0.07793982,  0.01006146,  0.02774708, -0.02058102],
      dtype=float32)

In [5]:
# get a single (most similar) result, translate it into the pydantic model
search_query.limit(1).to_pydantic(Words)[0].text

'hello world'

In [6]:
query = "farewell"
result = table.search(query).limit(1).to_pydantic(Words)[0]
print(result.text)

goodbye world


In [7]:
table.search(query).limit(1).to_pydantic(Words)

[Words(text='goodbye world', vector=FixedSizeList(dim=384))]

In [8]:
query = "random word"
result = table.search(query).limit(1).to_pydantic(Words)[0]
print(result.text)

hello world


Llamabot provides a class called `QueryBot` which implements everything above for you and allows you to just query the vector database.

So let's first write some "memories" as documents:

In [16]:
memories = ['Ben is really bad at video games, but Friend is excellent.',
       'Friend is a pro skiier, but Ben is terrified.',]

memory_filenames = []

for i, m in enumerate(memories):
    # write a temporary file
    temp_file = tempfile.NamedTemporaryFile(
        delete=False, mode='w', suffix=f'_memory_{i}.txt')
    with open(temp_file.name, "w") as f:
        f.write(m)
    print(f"Memory {i} written to {temp_file.name}")
    memory_filenames.append(temp_file.name)

Memory 0 written to /tmp/tmp3bkbj3bt_memory_0.txt
Memory 1 written to /tmp/tmp2dai8llc_memory_1.txt


In [18]:
from llamabot import QueryBot
from pathlib import Path

friend_prompt = """Your name is Friend.  \
You are having a conversation with your close friend Ben. \
You and Ben are sarcastic and poke fun at one another. \
But you care about each other and support one another."""

query_completer = QueryBot(
    system_prompt=friend_prompt,
    model_name=f"ollama_chat/{sft_model}",
    collection_name="memories",
    document_paths=memory_filenames
)

# # note - you'll want to reset the collection 
# # if you want to replace existing memories
#query_completer.docstore.reset()


  0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
query = "Remember that time we played video games?"
print("Retrieved memory: ", 
      query_completer.docstore.retrieve(query, 1))

response = query_completer(query,
                n_results=1)

Retrieved memory:  ['Ben is really bad at video games, but Friend is excellent.']
Oh yeah, the good old days when I was so much better than you! You know what they say: "The best players are always the worst ones." But hey, it's still fun to reminisce about those glory days. What game were we playing last time?

In [20]:
query = "Remember when we went skiing?"
print("Retrieved memory: ", 
      query_completer.docstore.retrieve(query, 1))

response = query_completer(query,
                n_results=1)

Retrieved memory:  ['Friend is a pro skiier, but Ben is terrified.']
Oh, that was such a blast! I remember how excited I got to try it out for the first time. You were so brave too, even if you looked like you were about to faint at some point.

But let's be real here, Ben. Skiing is not your thing. It's like trying to ride a bike without training wheels - just ask any of my friends who've tried to teach me how to ski. I'm much better off on the slopes than in the snow.

You're so good at it though! You can do some incredible tricks and flips that make everyone else look like they're wearing rollerblades. But you know what? That's okay too, right?

I mean, who am I to judge? Skiing is just another adventure for me. It's all about the thrill of the descent and the adrenaline rush. And if you ever change your mind, we can always go back next winter!